<a href="https://colab.research.google.com/github/ryo-ponsan/LangChain-tools/blob/main/langchain_research_summarize_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 論文の概要を把握するテンプレート（非プログラマー向け）
**社内データはくれぐれも入力しないように**お願いします。

論文や資料の概要をさらっと掴みたい際に使えると思います。その後、詳細を確認したい場合、元論文を確認する使い方が良いと思います。
## ノートブックの流れ
1. 各種ライブラリインストール
2. 前準備
3. 読み込むPDFファイルの選択
4. OpenAIのAPI keyの設定
5. プロンプトの設定
6. 結果出力
7. 応用
8. LangChainのポテンシャル
9. 注意事項
10. 参考
---

# 1. 各種ライブラリインストール
セルをクリックして、shift+Enterを押し各セルを実行してください。または、ランタイムタブから、「すべてのセルを実行」を選択してください。最初はshift+Enterで徐々に進める方法がおすすめです。

In [ ]:
%%writefile requirements.txt
openai
chromadb
langchain
pypdf
tiktoken

Writing requirements.txt


In [ ]:
%pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 858.2/858.2 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.8/248.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.6/922.6 kB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
    

# 2. 前準備
インストールが完了したら、各種ライブラリをインポートします。これでChatGPTやLangChainの機能を呼び出すことができます。

In [ ]:
import os
import platform

import openai
import chromadb
import langchain

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader

GoogleDriveをマウントします。GoogleColabからGoogleDriveのフォルダにアクセスできるようにします。実行されると確認画面が表示されるので承認してください。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 3. 読み込むPDFファイルの選択
下記の場合、マイドライブ直下のColab Notebooks直下に、datasetフォルダを事前に作成し、PDFファイルを保存しています。**各自変更変えてください。**

In [ ]:
loader = PyPDFLoader("/content/drive/My Drive/Colab Notebooks/world_model_dataset/Modeling Task Uncertainty for Safe Meta-Imitation Learning(2022).pdf")

In [ ]:
pages = loader.load_and_split()

# 4. OpenAIのAPI key設定
[OpenAI API key](https://platform.openai.com/account/api-keys)を取得し、`os.environ["OPENAI_API_KEY"] = 'sk-?????????????????????????????????'`の`sk-?????????????????????????????????`を書き換えてください。  尚API keyは誰にも共有しないよう注意してください。
発行手順は[こちら](https://auto-worker.com/blog/?p=6988)を参考にしてください。

## 注意事項  
OpenAI API key発行ページの「Usage」タブから、OpenAI APIの使用状況、使用期限が確認できます。使用上限、使用期限が過ぎていた場合、同アカウントで使用するには課金してください。

In [ ]:
os.environ["OPENAI_API_KEY"] = 'sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
openai.api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [ ]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(pages, embedding=embeddings, persist_directory=".")
vectorstore.persist()

In [ ]:
pdf_qa = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)

# 5. プロンプト設定
PDFから、抽出したり、要約したり、chatGPTに指示したい項目を入力してください。今回の例の場合下記内容を順番に抽出します。
1. {title}の概要
2. {title}の原理
3. {title}の新規性
4. {title}の課題、将来性  

尚queryは聞きたい項目だけ増やすことが可能です。今回のテンプレートの場合、先行研究を聞くプロンプトquery5 を用意した場合は、`query_list = [query1, query2, query3, query4, query5]` と`query_title = ["概要", "原理", "新規性", "課題等","先行研究"]`と設定しなおすとスムーズです。

また事例では、`MVLidarNet`という単語を使いまわしたかったため、`title`変数を用意していますが、単語不要な場合は、下記のような内容にすれば汎用性が高いと思います。

```
query1 = "Please give us an overview of this reserch? Answer in Japanese."
query2 = "Please tell me the technical principles of featured technology in this paper step by step in list form.Answer in Japanese."
query3 = "Please let me know if there is any description about the novelty of featured technology in this paper.Answer in Japanese."
query4 = "Please tell me step-by-step if there are any descriptions about featured technology in this paper issues and future prospects.Answer in Japanese."
```

In [ ]:
# title = "NVRaderNet"

# query1 = f"Please give us an overview of the {title}? Answer in Japanese."
# query2 = f"Please tell me the technical principles of the {title} step by step in list form.Answer in Japanese."
# query3 = f"Please let me know if there is any description about the novelty of the {title} technology.Answer in Japanese."
# query4 = f"Please tell me step-by-step if there are any descriptions about the {title} technology problems and future prospects.Answer in Japanese."
query0 = "Why did they start this reserch?If this paper describes the issues related to the technology of this genre in the past, the issues and problems of previous research and previous research, and the significance of conducting this research, please extract information. step by step in list form. Answer in Japanese step by step in list form"
query1 = "Please give us an overview of this reserch step by step in list form. Answer in Japanese step by step in list form."
query2 = "Please tell me the technical principles of featured technology in this paper step by step in list form. Answer in Japanese step by step in list form."
query3 = "Please let me know if there is any description about the novelty of featured technology in this paper step by step in list form. Answer in Japanese."
query4 = "Please tell me if there are any descriptions about issues of featured technology in this paper and future prospects step by step in list form.Answer in Japanese."
query5 = "Please tell me if there are any conclusion of featured technology in this paper and future prospects step by step in list form.Answer in Japanese within 50 words."

query_list = [query0, query1, query2, query3, query4, query5]
query_title = ["背景","概要", "原理", "新規性", "課題等","結論"]

# 6. 結果出力
PDFに記載されてない内容に関するプロンプトやあいまいなプロンプトを投げると、嘘(ハルシネーション)をいう可能性があります。そういった場合は、情報がない場合の回答方法を指示したり、ルールを設定することで回避可能です。

In [ ]:
all_result = ""
for i, query in enumerate(query_list):
    chat_history = []

    result = pdf_qa({"question": query, "chat_history": chat_history})
    print("############")
    print(f"{i+1} : {query_title[i]}")
    print("############")
    print(result["answer"])
    all_result += result["answer"]

############
1 : 背景
############
1. ロボットの汎用性の開発は、ロボット工学における重要な課題である。
2. ロボットが多様な環境で幅広いタスクを実行できるようにするため、データ駆動型アプローチが注目されている。
3. 特に、メタラーニングは、ロボット制御のメタラーニングの研究において、新しいタスクを解決するために他のタスクの経験を活用することができることが示されている。
4. しかし、これまでの研究では、パフォーマンスの向上に焦点が当てられており、安全性の問題は十分に探究されていない。
5. 本研究では、メタラーニングにおけるタスク不確実性と安全性の関係を明らかにし、安全なメタイミテーション学習のためのタスク不確実性の推定フレームワークを提案する。
############
2 : 概要
############
1. ロボットの柔軟性を高めるために、経験データからコントローラを学習することが有望なアプローチである。
2. メタ学習は、ロボットの制御を学習するための最も有望な新しい手法の1つであり、転移学習の中でも注目されている。
3. メタ学習において、タスクの不確実性を評価することは、安全なロボット学習システムの展開にとって重要である。
4. 本研究では、PETNetと呼ばれる新しいフレームワークを提案し、確率的タスク埋め込み空間でタスクの不確実性を推定することができる。
5. PETNetは、視覚的な模倣学習のためのタスク不確実性の推定において、良好な性能を発揮することが示された。
6. PETNetは、先行研究の方法と比較して、メタテストにおいて同等またはより高いパフォーマンスを達成することができる。
7. PETNetは、異常なデモンストレーションや合成されたデモンストレーションに対しても、タスクの不確実性を捉えることができる。
8. PETNetは、ロボット学習システムの安全な展開に向けた重要な一歩である。
############
3 : 原理
############
1. この論文の特徴的な技術は、PETNetと呼ばれるタスク不確実性を推定するためのフレームワークである。
2. PETNetは、タスク埋め込み空間で確率的推論を行うことにより、タスク不確実性を推定する。
3. PETNetは、メタ模倣学習のベンチマ

## 6-2. パワポ用に出力を修正(テスト用)



In [ ]:
# query = f"Summarize the contents enclosed in ``` ``` in {len(query_title)} chapter format step by step in list form by using number.```{all_result}``` Answer in Japanese."

# chat_history = []

# result = pdf_qa({"question": query, "chat_history": chat_history})

# print(result["answer"])

# 7. 応用

上記は、それぞれが独立したqueryになっていましたが、過去の質疑回答を踏まえた上で、chatGPTが回答する方法や、chatBotのような対話形式で形で回答する方法、など様々な方法があります。下記過去の質疑回答を踏まえた上で回答させる一例です。試したい方は、下の方にコードとして書き込んでみてください。PDFは最初に読み込んだものを参照しています。[こちらの記事](https://zenn.dev/umi_mori/books/prompt-engineer/viewer/langchain_overview)も参考になります。




### 1回目のやり取り

```

query = "what's the main purpose of this article? Answer in Japanese."

chat_history = []

result = pdf_qa({"question": query, "chat_history": chat_history})

print(result["answer"])

```

### 1回目に対する2回目のやり取り

```

query2 = "Please tell me more in detail.Answer in Japanese."
chat_history1 = [(query, result["answer"])]

result2 = pdf_qa({"question": query2, "chat_history": chat_history1})

print(result2["answer"])


```

# 8.LangChainのポテンシャル
PDFをchatGPTに読ませた上で、情報抽出することができました。ほかにもchatGPTや他のLLMモデルにGoogle検索を使ってウェブスクレイピングをさせたり、chatGPTとの対話記録を保持するデータベースを使い対話型にしたり、といったことがコード数行で実現できるようになっているため、業務効率化に繋がり、ポテンシャルが非常に高いライブラリだと思います。皆さん使っていきましょう。

# 9.注意事項
1. **(再掲)社外秘はくれぐれも入力しないように**お願いします（現状、chatGPT API を使うとOpenAIにデータが飛んでしまうため）。
2. OpenAI のAPIには無課金の場合、使用上限、使用期限があります。無料枠がなくなったら、課金しましょう、するだけの価値はありそうです。

# 10.参考
- 気になる方は[LangChainの公式ドキュメント](https://python.langchain.com/en/latest/index.html)があります。
- 本ノートブックは、[こちらの記事](https://qiita.com/hiroki_okuhata_int/items/7102bab7d96eb2574e7d)を参考にしました。
- プロンプトの書き方のコツについては、[こちらのサイト](https://www.promptingguide.ai/jp)が参考になります。